In [1]:
import os

In [2]:
%pwd

'/workspaces/stok_opt-mimizasyon_mlops_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/stok_opt-mimizasyon_mlops_project'

# ENTİTİY


In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    xgboost_params: Dict[str, Any]  # XGBoost parametrelerini burada saklayacağız

# CONFİG

In [6]:
from src.stok_optimizasyonu_ml_project.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.stok_optimizasyonu_ml_project.utils.common import read_yaml,create_directories

In [7]:
# ConfigurationManager sınıfı
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        # YAML dosyalarını okuma
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)  # params.yaml
        self.schema = read_yaml(schema_filepath)  # schema.yaml

        # Eğer root_dir bir str ise, Path ile dönüştürülmeli
        self.config.artifacts_root = Path(self.config.artifacts_root)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        # Config, params ve schema'ya erişim sağlıyoruz
        config = self.config.model_trainer
        params = self.params.xgboost  # params.yaml'dan XGBoost parametreleri
        schema = self.schema.TARGET_COLUMN  # target_column bilgisi schema.yaml'dan

        # Train ve test dataset yolları artifacts altında olacak şekilde ayarlanmalı
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,  # Target sütunu
            xgboost_params=params,  # XGBoost parametrelerini geçiyoruz
        )
        
        return model_trainer_config

# COMPONENT

In [8]:
import pandas as pd
import os
from src.stok_optimizasyonu_ml_project import logger
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import joblib
from sklearn.metrics import mean_squared_error


In [9]:
from sklearn.model_selection import train_test_split

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.model = None

    def train(self):
        # Eğitim ve test verilerini yükle
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        # Eğitim verisinden örnekleme yap
        train_data_sample = train_data.sample(frac=0.1, random_state=42)  # Verinin %10'u ile eğitim
        
        # X ve y'yi ayır
        X_train = train_data_sample.drop(self.config.target_column, axis=1)
        y_train = train_data_sample[self.config.target_column]
        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[self.config.target_column]

        # XGBoost modelini oluştur
        xgb_model = xgb.XGBRegressor(
            objective=self.config.xgboost_params['objective'],
            booster=self.config.xgboost_params['booster']
        )

        # Parametre ızgarası
        param_grid = {
            'n_estimators': self.config.xgboost_params['n_estimators'],
            'max_depth': self.config.xgboost_params['max_depth'],
            'learning_rate': self.config.xgboost_params['learning_rate'],
            'subsample': self.config.xgboost_params['subsample'],
            'colsample_bytree': self.config.xgboost_params['colsample_bytree'],
            'gamma': self.config.xgboost_params['gamma'],
            'reg_alpha': self.config.xgboost_params['reg_alpha'],
            'reg_lambda': self.config.xgboost_params['reg_lambda']
        }

        # GridSearchCV ile parametre araması yapalım
        grid_search = GridSearchCV(xgb_model, param_grid, cv=3, verbose=1)
        grid_search.fit(X_train, y_train)

        # En iyi parametreyi al
        best_model = grid_search.best_estimator_

        # Modeli test et
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        print(f'Mean Squared Error: {mse}')

        # Modeli kaydet
        joblib.dump(best_model, self.config.root_dir / self.config.model_name)

In [10]:
try:
    # ConfigurationManager ile model konfigürasyonlarını alıyoruz
    config = ConfigurationManager()
    
    # ModelTrainerConfig nesnesini alıyoruz
    model_trainer_config = config.get_model_trainer_config()

    # ModelTrainer sınıfını oluşturuyoruz, model_trainer_config parametre olarak geçiliyor
    model_trainer = ModelTrainer(config=model_trainer_config)

    # Modeli eğitiyoruz
    model_trainer.train()

except FileNotFoundError as e:
    # Dosya bulunamadığında detaylı hata mesajı
    raise Exception(f"Dosya bulunamadı: {e}")

except KeyError as e:
    # Anahtar hatası durumunda, eksik anahtar ile ilgili mesaj
    raise Exception(f"YAML dosyasındaki eksik anahtar: {e}")

except Exception as e:
    # Diğer genel hatalar için istisna bilgisi
    raise Exception(f"Bir hata oluştu: {e}")

Fitting 3 folds for each of 375000 candidates, totalling 1125000 fits


KeyboardInterrupt: 